Task 1: Batch Processing Simulation (Python)
Write a Python script to execute multiple .py files sequentially, mimicking batch processing.

In [ ]:
#!/usr/bin/env python3

import subprocess
import os

def run_scripts_sequentially(scripts):
    for script in scripts:
        if not os.path.exists(script):
            print(f"[SKIP] {script} not found.")
            continue

        print(f"\n[INFO] Executing {script} ...")
        result = subprocess.call(['python3', script])

        if result == 0:
            print(f"[OK] {script} executed successfully.")
        else:
            print(f"[ERROR] {script} exited with status {result}")

if __name__ == "__main__":
    scripts_to_run = ['script1.py', 'script2.py', 'script3.py']
    print("[BATCH] Starting batch processing...")
    run_scripts_sequentially(scripts_to_run)
    print("\n[BATCH] Batch processing finished.")


Executing script1.py...
Executing script2.py...
Executing script3.py...


Task 2: System Startup and Logging
Simulate system startup using Python by creating multiple processes and logging their start and end into a log file.


In [ ]:
#!/usr/bin/env python3

import multiprocessing
import logging
import time
import random

LOG_FILE = "system_log.txt"
logging.basicConfig(
    filename=LOG_FILE,
    level=logging.INFO,
    format="%(asctime)s - %(processName)s - %(message)s"
)

def process_task(name, work_time):
    logging.info(f"{name} started")
    time.sleep(work_time)
    logging.info(f"{name} terminated")

if __name__ == '__main__':
    print("System Booting...")

    processes = []
    for i in range(1, 4):
        p_name = f"Service-{i}"
        wt = random.randint(1, 4)
        p = multiprocessing.Process(target=process_task, args=(p_name, wt), name=p_name)
        processes.append(p)

    for p in processes:
        p.start()

    for p in processes:
        p.join()

    print("System Shutdown.")
    print(f"Logs written to {LOG_FILE}")



System Booting...
System Shutdown.


Task 3: System Calls and IPC (Python - fork, exec, pipe)
Use system calls (fork(), exec(), wait()) and implement basic Inter-Process Communication using pipes in C or Python.


In [ ]:

import os

def parent_process(write_fd):
    os.close(read_fd)
    message = "Hello from parent via pipe!"
    print("[PARENT] Sending message to child...")
    os.write(write_fd, message.encode())
    os.close(write_fd)
    pid, status = os.wait()
    print(f"[PARENT] Child {pid} exited with status {status}")

def child_process(read_fd):
    os.close(write_fd)
    data = os.read(read_fd, 1024)
    print("[CHILD] Received from parent:", data.decode())
    os.close(read_fd)

    print("[CHILD] Executing 'ls -l' using exec...")
    os.execlp("ls", "ls", "-l")

if __name__ == "__main__":
    read_fd, write_fd = os.pipe()
    pid = os.fork()

    if pid > 0:
        parent_process(write_fd)
    else:
        child_process(read_fd)


[CHILD] Received from parent: Hello from parent via pipe!
[CHILD] Executing 'ls -l' using exec...
[PARENT] Sending message to child...
[PARENT] Child 23181 exited with status 0


/tmp/ipython-input-3157429053.py:23: DeprecationWarning: This process (pid=286) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()


/*
 Task 3: System Calls and IPC in C

 Demonstrate:
 - fork()
 - pipe()
 - write(), read()
 - wait()
 - execvp()
*/

In [ ]:
%%writefile ipc_c.c
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <sys/wait.h>
#include <string.h>

int main() {
    int fd[2];  // fd[0] = read end, fd[1] = write end
    pid_t pid;
    char buffer[100];

    if (pipe(fd) == -1) {
        perror("pipe");
        return 1;
    }

    pid = fork();

    if (pid < 0) {
        perror("fork");
        return 1;
    }

    if (pid > 0) {
        // Parent process
        close(fd[0]);  // close read end
        const char *msg = "Hello from parent via pipe (C)!";
        write(fd[1], msg, strlen(msg) + 1);
        close(fd[1]);  // done writing

        // Wait for child
        wait(NULL);
        printf("[PARENT] Child finished execution.\n");
    } else {
        // Child process
        close(fd[1]);  // close write end
        read(fd[0], buffer, sizeof(buffer));
        close(fd[0]);

        printf("[CHILD] Received: %s\n", buffer);

        // Now use execvp to run "ls -l"
        printf("[CHILD] Executing 'ls -l' using execvp...\n");
        char *args[] = {"ls", "-l", NULL};
        execvp("ls", args);

        // Only executed if execvp fails
        perror("execvp failed");
        exit(1);
    }

    return 0;
}

!gcc ipc_c.c -o ipc_c
!./ipc_c

Overwriting ipc_c.c


Task 4: VM Detection and Shell Interaction
Create a shell script to print system details and a Python script to detect if the system is running inside a virtual machine.


In [ ]:
#!/usr/bin/env python3
"""
Task 4: VM Detection in Python

Heuristics:
- Check common files for virtualization info
- Read /sys/class/dmi/id/product_name and sys_vendor
- Look for known VM vendor strings (VirtualBox, VMware, KVM, etc.)
"""

import os

def read_file(path):
    try:
        with open(path, 'r') as f:
            return f.read().strip()
    except FileNotFoundError:
        return None
    except PermissionError:
        return None

def detect_vm():
    indicators = []

    product_name = read_file("/sys/class/dmi/id/product_name")
    sys_vendor = read_file("/sys/class/dmi/id/sys_vendor")

    if product_name:
        indicators.append(f"product_name={product_name}")
    if sys_vendor:
        indicators.append(f"sys_vendor={sys_vendor}")

    vm_keywords = ["VirtualBox", "VMware", "KVM", "QEMU", "Hyper-V", "Xen"]

    is_vm = False
    joined = " ".join(indicators)

    for keyword in vm_keywords:
        if keyword.lower() in joined.lower():
            is_vm = True
            break

    try:
        import subprocess
        result = subprocess.run(
            ["systemd-detect-virt"],
            stdout=subprocess.PIPE,
            stderr=subprocess.DEVNULL,
            text=True
        )
        output = result.stdout.strip()
        if output not in ("", "none"):
            indicators.append(f"systemd-detect-virt={output}")
            is_vm = True
    except Exception:
        pass

    return is_vm, indicators

if __name__ == "__main__":
    is_vm, indicators = detect_vm()
    print("===== VM Detection Report =====")
    print("Indicators found:")
    if indicators:
        for item in indicators:
            print(" -", item)
    else:
        print(" - No indicators found")

    if is_vm:
        print("\nConclusion: System is LIKELY running inside a Virtual Machine.")
    else:
        print("\nConclusion: System is LIKELY running on bare metal (no strong VM indicators).")


===== VM Detection Report =====
Indicators found:
 - product_name=Google Compute Engine
 - sys_vendor=Google
 - systemd-detect-virt=docker

Conclusion: System is LIKELY running inside a Virtual Machine.


Task 5: CPU Scheduling Algorithms
Implement FCFS, SJF, Round Robin, and Priority Scheduling algorithms in Python to calculate WT and TAT.


In [ ]:

from typing import List, Dict

def print_results(title: str, processes: List[Dict]):
    print(f"\n===== {title} =====")
    print("PID\tAT\tBT\tPR\tWT\tTAT")
    total_wt = 0
    total_tat = 0

    for p in processes:
        total_wt += p["wt"]
        total_tat += p["tat"]
        pr = p.get("priority", "-")
        print(f"{p['pid']}\t{p['at']}\t{p['bt']}\t{pr}\t{p['wt']}\t{p['tat']}")

    n = len(processes)
    print(f"\nAverage WT  = {total_wt / n:.2f}")
    print(f"Average TAT = {total_tat / n:.2f}")


# ---------- FCFS ----------
def fcfs(processes: List[Dict]):
    procs = sorted(processes, key=lambda x: x["at"])
    current_time = 0

    for p in procs:
        if current_time < p["at"]:
            current_time = p["at"]
        p["wt"] = current_time - p["at"]
        current_time += p["bt"]
        p["tat"] = p["wt"] + p["bt"]

    print_results("FCFS", procs)


# ---------- SJF (Non-preemptive) ----------
def sjf_non_preemptive(processes: List[Dict]):
    procs = [p.copy() for p in processes]  # safe copy
    n = len(procs)
    completed = 0
    current_time = 0
    visited = [False] * n

    while completed < n:
        idx = -1
        min_bt = float("inf")

        for i in range(n):
            if (procs[i]["at"] <= current_time) and not visited[i]:
                if procs[i]["bt"] < min_bt:
                    min_bt = procs[i]["bt"]
                    idx = i

        if idx == -1:
            current_time += 1
            continue

        p = procs[idx]
        visited[idx] = True
        current_time += p["bt"]
        p["tat"] = current_time - p["at"]
        p["wt"] = p["tat"] - p["bt"]
        completed += 1

    print_results("SJF (Non-preemptive)", procs)


# ---------- Priority (Non-preemptive) ----------
def priority_non_preemptive(processes: List[Dict]):
    procs = [p.copy() for p in processes]
    n = len(procs)
    completed = 0
    current_time = 0
    visited = [False] * n

    while completed < n:
        idx = -1
        best_pr = float("inf")

        for i in range(n):
            if (procs[i]["at"] <= current_time) and not visited[i]:
                if procs[i]["priority"] < best_pr:
                    best_pr = procs[i]["priority"]
                    idx = i

        if idx == -1:
            current_time += 1
            continue

        p = procs[idx]
        visited[idx] = True
        current_time += p["bt"]
        p["tat"] = current_time - p["at"]
        p["wt"] = p["tat"] - p["bt"]
        completed += 1

    print_results("Priority (Non-preemptive)", procs)


def round_robin(processes: List[Dict], quantum: int):
    procs = [p.copy() for p in processes]
    n = len(procs)

    for p in procs:
        p["remaining_bt"] = p["bt"]

    current_time = 0
    completed = 0
    queue = []
    visited = [False] * n

    def add_new_arrivals():
        for i in range(n):
            if (procs[i]["at"] <= current_time) and not visited[i]:
                queue.append(i)
                visited[i] = True

    add_new_arrivals()

    while completed < n:
        if not queue:
            current_time += 1
            add_new_arrivals()
            continue

        idx = queue.pop(0)
        p = procs[idx]

        if p["remaining_bt"] > quantum:
            current_time += quantum
            p["remaining_bt"] -= quantum
            add_new_arrivals()
            queue.append(idx)
        else:
            current_time += p["remaining_bt"]
            p["remaining_bt"] = 0
            p["tat"] = current_time - p["at"]
            p["wt"] = p["tat"] - p["bt"]
            completed += 1
            add_new_arrivals()

    print_results(f"Round Robin (q={quantum})", procs)

if __name__ == "__main__":
    processes = [
        {"pid": "P1", "at": 0, "bt": 5, "priority": 2},
        {"pid": "P2", "at": 1, "bt": 3, "priority": 1},
        {"pid": "P3", "at": 2, "bt": 8, "priority": 3},
        {"pid": "P4", "at": 3, "bt": 6, "priority": 2},
    ]

    fcfs(processes)
    sjf_non_preemptive(processes)
    priority_non_preemptive(processes)
    round_robin(processes, quantum=2)



===== FCFS =====
PID	AT	BT	PR	WT	TAT
P1	0	5	2	0	5
P2	1	3	1	4	7
P3	2	8	3	6	14
P4	3	6	2	13	19

Average WT  = 5.75
Average TAT = 11.25

===== SJF (Non-preemptive) =====
PID	AT	BT	PR	WT	TAT
P1	0	5	2	0	5
P2	1	3	1	4	7
P3	2	8	3	12	20
P4	3	6	2	5	11

Average WT  = 5.25
Average TAT = 10.75

===== Priority (Non-preemptive) =====
PID	AT	BT	PR	WT	TAT
P1	0	5	2	0	5
P2	1	3	1	4	7
P3	2	8	3	12	20
P4	3	6	2	5	11

Average WT  = 5.25
Average TAT = 10.75

===== Round Robin (q=2) =====
PID	AT	BT	PR	WT	TAT
P1	0	5	2	9	14
P2	1	3	1	7	10
P3	2	8	3	12	20
P4	3	6	2	11	17

Average WT  = 9.75
Average TAT = 15.25
